# Generate model predictions

Generate the predictions of the continuous model

### Imports and setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=4)
from functions_continuous_model import time_evolution
from system_definition import T, Lambda_0, tau, omega, s

%load_ext autoreload
%autoreload 2

### Define the parameters of the model

In [5]:
# Folder to save the results
folder = 'RESULTS/'

# Basic parameters
R0 = 1.5
symptomatics = 0.8
testing = 0.0
delay = 2

# Custom R0 used to run different simulations
def beta(tau, R0):
    return R0 * omega(tau)

# Custom epsilon used to run different simulations
def eps(tau, e_i, e_t):
    eps_I = e_i + 0 * tau
    eps_T = e_t + 0 * tau
    return eps_I, eps_T

# Size of the discretization grid for (eps_I, eps_T)
n_i = 10
n_t = 10

### Run the time evolution

In [3]:
ep_i = np.linspace(0, 1, n_i)    
ep_t = np.linspace(0, 1, n_t)    

tail_length = 10

end_value = np.zeros((n_i, n_t))

for i in range(n_i):
    for j in range(n_t):
        print('Running simulation [ % 4d / % 4d] ---- (ep_i, ep_t) = (%2.2e, %2.2e)' 
              % (1 + i * n_t + j, n_i * n_t, ep_i[i], ep_t[j]))
        lambda_ , _, _ = time_evolution(tau, Lambda_0, T, 
                                        epsilon=lambda tau: eps(tau, ep_i[i], ep_t[j]),
                                        beta=lambda tau: beta(tau, R0=R0),
                                        s=lambda tau: s(tau, symptomatics=symptomatics, testing=testing, delay=delay))
        R_t = lambda_[1:] / lambda_[:-1]
        end_value[i, j] = np.mean(R_t[-tail_length:])


end_value = (end_value - 1) * 100  

Running simulation [    1 /  100] ---- (ep_i, ep_t) = (0.00e+00, 0.00e+00)
Running simulation [    2 /  100] ---- (ep_i, ep_t) = (0.00e+00, 1.11e-01)
Running simulation [    3 /  100] ---- (ep_i, ep_t) = (0.00e+00, 2.22e-01)
Running simulation [    4 /  100] ---- (ep_i, ep_t) = (0.00e+00, 3.33e-01)
Running simulation [    5 /  100] ---- (ep_i, ep_t) = (0.00e+00, 4.44e-01)
Running simulation [    6 /  100] ---- (ep_i, ep_t) = (0.00e+00, 5.56e-01)
Running simulation [    7 /  100] ---- (ep_i, ep_t) = (0.00e+00, 6.67e-01)
Running simulation [    8 /  100] ---- (ep_i, ep_t) = (0.00e+00, 7.78e-01)
Running simulation [    9 /  100] ---- (ep_i, ep_t) = (0.00e+00, 8.89e-01)
Running simulation [   10 /  100] ---- (ep_i, ep_t) = (0.00e+00, 1.00e+00)
Running simulation [   11 /  100] ---- (ep_i, ep_t) = (1.11e-01, 0.00e+00)
Running simulation [   12 /  100] ---- (ep_i, ep_t) = (1.11e-01, 1.11e-01)
Running simulation [   13 /  100] ---- (ep_i, ep_t) = (1.11e-01, 2.22e-01)
Running simulation [   14

### Save the results

In [6]:
# File where to save the results
base_name = 'critical_values_asympt_20_R0_%2.2f_testing_%2.2f_delay_%2.d_ni_%2.d_nt_%2.d.csv' % (R0, testing, delay, n_i, n_t)
  
np.savetxt(folder + base_name, end_value, delimiter=',')